### About this file

This file uses OpenAI to determine whether Cambridge, MA 311 reports are about rats, using the open text responses submitted by residents.

Ultimately, due to time and expense, it was faster and similarly accurate to use regular expressions to determine if a 311 reports was rat-related.

In [1]:
import files
import os
import pandas as pd
from openai import OpenAI
from IPython.display import display, Markdown
from tqdm.auto import tqdm # makes pretty progress bars
import re
tqdm.pandas()

from dotenv import load_dotenv

In [159]:
load_dotenv()
api_key = os.environ['MyDevelopmentKey']

In [160]:
df = pd.read_csv("../data/Cambridge_MA_Commonwealth_Connect_Service_Requests_20250729.csv")

In [161]:
# Goal: Determine what number of issue types with the substring "rat" are really about rats!
df['issue_text_full'] = df['issue_type'] + '. ' + df['issue_description']

rat_related = df[
    df['issue_text_full'].str.contains(r'\brats?\b', case=False, na=False, regex=True) &
    (df['issue_category'] != "Rodent Sighting")
]


In [162]:
rat_related_sample = rat_related.sample(n=50)

In [163]:
# set up connection to OpenAI client
client = OpenAI(api_key=api_key)

In [164]:
from enum import Enum, IntEnum

class RatResponseOptions(str, Enum):
    about_rats = "About rats"
    not_about_rats = "Not about rats"

# Be sure to list the categories with the magic phrase {categories}
prompt_base = """

You are a talented rodent infestation expert and analyst.

You are tasked with determining whether 311 incident reports are about Rats or not.

I'm going to give you open-text responses submitted by residents and you will classify them into a category.

Give the best match for the category. Return only a single category.

Choose only from these options: {categories}

Here is the idea:

{idea_text}

"""

In [165]:
# for convenience, make a column in our dataframes with the prompt for each item
rat_related_sample_prompt_column = rat_related_sample.apply(lambda row: prompt_base.format(
    idea_text = row["issue_text_full"],
    categories=", ".join(['"{}"'.format(opt.value) for opt in RatResponseOptions])
), axis="columns")

rat_related_full_prompt_column = rat_related.apply(lambda row: prompt_base.format(
    idea_text = row["issue_text_full"],
    categories=", ".join(['"{}"'.format(opt.value) for opt in RatResponseOptions])
), axis="columns")

In [166]:
## Cost estimation

from strip_tags import strip_tags
import tiktoken

def count_tokens(model, text):
    encoding = tiktoken.encoding_for_model(model if model != 'gpt-4.1-mini' else 'gpt-4o')
    tokens = encoding.encode(text)
    return len(tokens)

input_token_costs = {
    "gpt-4.1-mini": 0.4 / 1_000_000,  # https://openai.com/api/pricing/
    "mistral-medium-latest": 0.4 / 1_000_000, # https://mistral.ai/pricing#api-pricing
    "mistral-large-latest": 2.0 / 1_000_000
}
def estimate_cost(model, token_count):
    return token_count * input_token_costs[model]

MODEL_TO_USE = 'gpt-4.1-mini'

count_tokens_for_our_model = lambda text: count_tokens(MODEL_TO_USE, text)

In [167]:
# Cost estimation for sample dataset
token_count_sample = count_tokens_for_our_model("SAMPLE RESPONSE SAMPLE RESPONSE".join(rat_related_sample_prompt_column))
"Sample would cost: ${:.2f}".format(estimate_cost(MODEL_TO_USE, token_count_sample))

'Sample would cost: $0.00'

In [168]:
## cost estimation for full dataset
token_count_full = count_tokens_for_our_model("SAMPLE RESPONSE SAMPLE RESPONSE".join(rat_related_full_prompt_column))
"Full dataset would cost: ${:.2f}".format(estimate_cost(MODEL_TO_USE, token_count_full))

'Full dataset would cost: $0.11'

In [169]:
# Function to actually send the prompt to OpenAI and get the answer
from pydantic import BaseModel
import time

class RatValidCategories(BaseModel):
  classification: RatResponseOptions

def classify(prompt_including_idea):

    # put our prompt into the blob that OpenAI expects
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": prompt_including_idea,
        }
    ]

    chat_completion = client.responses.parse(
        input=messages,
        model='gpt-4o',
        text_format=RatValidCategories,
    )
    # get the answer out of the blob that OpenAI returns.
    resp = chat_completion.output_parsed.classification.value if chat_completion.output_parsed else None
    return resp

In [170]:
rat_related_sample["ai_guess"] = rat_related_sample_prompt_column.progress_apply(classify)

  0%|          | 0/50 [00:00<?, ?it/s]

In [171]:
display(Markdown("## Here are a few results. How good did we do?"))
with pd.option_context("display.max_colwidth", 500):
  display(
      rat_related_sample[["issue_text_full", "ai_guess"]].head(10)
  )

## Here are a few results. How good did we do?

,issue_text_full,ai_guess
111480,Roadkill / Dead Animal. Dead rat.,About rats
15640,Overflowing Public Trash/Recycling Receptacle. Overflowing dumpster in neighborhood dealing with a lot of rats due to mismanaged dumpsters. Please address.,About rats
13657,Rodent Sighting. Dead rat in front of the door of 345 Franklin street,About rats
3050,Other. The trash here is again not handled properly I have seen rats and will provide photos as soon as I can but the bags of trash are clearly visible from the street.,About rats
6016,"Overflowing Public Trash/Recycling Receptacle. Trash bags not in a bin left overnight for the rats to eat, again. I'm starting to wonder if DPW is telling the Frisoli Youth Center to leave their trash here. Either way, it's happening on a regular basis, DPW could you please install a bin?",About rats
54822,Roadkill / Dead Animal. Dead rat,About rats
78956,Roadkill / Dead Animal. dead rat in St 43 market,About rats
13371,Overflowing Public Trash/Recycling Receptacle. Need dog waste bin instead of recycling here. The big belly trash on corner near tennis court is constantly left open ((almost daily)) and I'm terrified to go near it because of the amount of rats I see in the area.,About rats
76894,Roadkill / Dead Animal. Dead rat. Please be informed.,About rats
1808,Rodent Sighting. Dead rat on sidewalk - pls remove asap,About rats


In [144]:
# Export CSV for manual classification/groundtruthing

# rat_related_sample["groundtruth"] = ""
# rat_related_sample.to_csv('../data/cambridge_rat_311_for_hand_classification.csv')

In [148]:
# handcoded = pd.read_csv("../data/handcoded.csv")
# from sklearn.metrics import accuracy_score
# display(Markdown("Accuracy score: {:.1%}. Is that good?".format(
#     accuracy_score(handcoded["groundtruth"], handcoded["ai_guess"])
# )))

Accuracy score: 98.0%. Is that good?

In [146]:
# are_you_sure_you_want_to_classify_the_whole_thing = input()

In [147]:
# rat_related["ai_guess"] = rat_related_full_prompt_column.progress_apply(classify)

In [172]:
from tqdm import tqdm

# Make sure your DataFrame index matches the Series
rat_related_export = rat_related.copy().reset_index()  # in case you're slicing views

results = []

# Loop through the prompts with index
for i, (idx, prompt) in tqdm(enumerate(rat_related_full_prompt_column.items()), total=len(rat_related_full_prompt_column)):
    try:
        guess = classify(prompt)
    except Exception as e:
        guess = None
        print(f"Error at i {i} index {idx}: {e}")
    
    results.append((idx, guess))

    # Save progress every 10 entries
    if i % 10 == 0:
        for j, g in results:
            rat_related_export.loc[j, "ai_guess"] = g
        rat_related_export.to_csv("../data/rat_related_partial.csv", index=False)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2328/2328 [49:09<00:00,  1.27s/it]


In [173]:
rat_related_export.to_csv("../data/rat_related_full.csv", index=False)